In [1]:
import json
import numpy as np
import evaluate

2024-12-17 21:48:25.927155: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-17 21:48:26.742524: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# orchestrator_file_path = '/accounts/grad/phudish_p/294-196-moa/finetune_pipeline/evaluation_results_2.json'
orchestrator_file_path = '/accounts/grad/phudish_p/294-196-moa/finetune_pipeline/evaluation_results_baseline_2.json'

results_orchestrator = []
with open(orchestrator_file_path, 'r') as f:
    for line in f:
        try:
            result = json.loads(line)
            results_orchestrator.append(result)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line: {line}")
            print(e)

print(f"Number of results: {len(results_orchestrator)}")

Number of results: 1080


In [3]:
# architecture_file_path = "/accounts/grad/phudish_p/294-196-moa/finetune_pipeline/evaluation_results_architecture.json"
architecture_file_path = "/accounts/grad/phudish_p/294-196-moa/finetune_pipeline/evaluation_results_architecture_2.json"

architecture_results = []
with open(architecture_file_path, 'r') as f:
    for line_number, line in enumerate(f, 1):
        line = line.strip()
        if not line.startswith('{'):
            continue  # Skip lines that do not start with a JSON object
        try:
            result = json.loads(line)
            architecture_results.append(result)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line {line_number}: {line}")
            print(e)
print(f"Number of results: {len(architecture_results)}")

Number of results: 1080


In [4]:
print(f"Average inference time baseline orchestrator: {np.mean([result['inference_time'] for result in results_orchestrator])}")
print(f"Average inference time architecture: {np.mean([result['inference_time'] for result in architecture_results])}")

Average inference time baseline orchestrator: 10.087056978543599
Average inference time architecture: 7.315534818393213


In [5]:
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /accounts/grad/phudish_p/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
invalid_count = 0
line_number_invalid = []
bleu_score_list = []
rogue_list = []
rogueL_list = []
inference_time_list = []
met_score_list = []

for line_number, result in enumerate(architecture_results, 1):
    query = result['query']
    target_text = result['target_text']
    answer = result['response']
    answer_only = answer.find(query) + len(query)
    inf_time = result['inference_time']
    # print(f"Query: {query}")
    #print(f"Answer only : {answer[answer_only:]}")
    try:
        bleu_score = bleu.compute(predictions=[answer[answer_only:]], references=[target_text])
        out_tok =  word_tokenize(answer[answer_only:])
        ref_tok = word_tokenize(target_text)       
        met_score = meteor_score([ref_tok], out_tok)
        met_score_list.append(met_score)
        inference_time_list.append(inf_time)
        print(f"BLEU score: {bleu_score}")
        print(f"Met score: {met_score}")
        
    except Exception as e:
        # print(f"Error computing BLEU score on line {line_number}")
        print(e)    
        invalid_count += 1
        line_number_invalid.append(line_number)
        #print(f"Answer only : {answer[answer_only:]}")

BLEU score: {'bleu': 0.0, 'precisions': [0.18947368421052632, 0.010638297872340425, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.5833333333333333, 'translation_length': 95, 'reference_length': 60}
Met score: 0.20086533832732564
BLEU score: {'bleu': 0.05413546863043105, 'precisions': [0.5333333333333333, 0.27586206896551724, 0.14285714285714285, 0.07407407407407407], 'brevity_penalty': 0.27253179303401254, 'length_ratio': 0.43478260869565216, 'translation_length': 30, 'reference_length': 69}
Met score: 0.20176130389064148
BLEU score: {'bleu': 0.0, 'precisions': [0.56, 0.125, 0.043478260869565216, 0.0], 'brevity_penalty': 0.4867522559599717, 'length_ratio': 0.5813953488372093, 'translation_length': 25, 'reference_length': 43}
Met score: 0.20190023752969122
BLEU score: {'bleu': 0.0, 'precisions': [0.375, 0.0, 0.0, 0.0], 'brevity_penalty': 0.25283959580474646, 'length_ratio': 0.42105263157894735, 'translation_length': 8, 'reference_length': 19}
Met score: 0.11173184357541899
BLEU s

In [7]:
invalid_count_orch = 0
line_number_invalid_orch = []
bleu_score_list = []
rogue_list = []
rogueL_list = []
inference_time_list_orch = []
met_score_list_orch = []

for line_number, result in enumerate(results_orchestrator, 1):
    query = result['query']
    target_text = result['target_text']
    answer = result['response']
    answer_only = answer.find(query) + len(query)
    inf_time = result['inference_time']
    # print(f"Query: {query}")
    #print(f"Answer only : {answer[answer_only:]}")
    try:
        bleu_score = bleu.compute(predictions=[answer[answer_only:]], references=[target_text])
        out_tok =  word_tokenize(answer[answer_only:])
        ref_tok = word_tokenize(target_text)       
        met_score = meteor_score([ref_tok], out_tok)
        met_score_list_orch.append(met_score)
        inference_time_list_orch.append(inf_time)
        print(f"BLEU score: {bleu_score}")
        print(f"Met score: {met_score}")
        
    except Exception as e:
        # print(f"Error computing BLEU score on line {line_number}")
        print(e)    
        invalid_count += 1
        line_number_invalid_orch.append(line_number)
        #print(f"Answer only : {answer[answer_only:]}")

BLEU score: {'bleu': 0.1631745783980777, 'precisions': [0.39285714285714285, 0.16363636363636364, 0.12962962962962962, 0.11320754716981132], 'brevity_penalty': 0.9310627797040228, 'length_ratio': 0.9333333333333333, 'translation_length': 56, 'reference_length': 60}
Met score: 0.26405300184413627
BLEU score: {'bleu': 0.28979770375168407, 'precisions': [0.6976744186046512, 0.5476190476190477, 0.4878048780487805, 0.425], 'brevity_penalty': 0.5462649509478217, 'length_ratio': 0.6231884057971014, 'translation_length': 43, 'reference_length': 69}
Met score: 0.37036800490502886
BLEU score: {'bleu': 0.3002418636265675, 'precisions': [0.5405405405405406, 0.3888888888888889, 0.3142857142857143, 0.23529411764705882], 'brevity_penalty': 0.8503033063369504, 'length_ratio': 0.8604651162790697, 'translation_length': 37, 'reference_length': 43}
Met score: 0.4265767223978367
BLEU score: {'bleu': 0.35071464521385104, 'precisions': [0.4827586206896552, 0.39285714285714285, 0.2962962962962963, 0.269230769

In [10]:
print(f"Invalid count: {invalid_count}")

print("Arch Results")
print(f"MET SCORE Min: {np.min(np.asarray(met_score_list))}, Max: {np.max(np.asarray(met_score_list))}, Mean:{np.mean(np.asarray(met_score_list))}, Median:{np.median(np.asarray(met_score_list))}")
print(f"Inference time Min: {np.min(np.asarray(inference_time_list))}, Max: {np.max(np.asarray(inference_time_list))}, Mean: {np.mean(np.asarray(inference_time_list))}")
print("\n")

print("Baseline Results ")
print(f"MET SCORE Min: {np.min(np.asarray(met_score_list_orch))}, Max: {np.max(np.asarray(met_score_list_orch))}, Mean:{np.mean(np.asarray(met_score_list_orch))}, Median:{np.median(np.asarray(met_score_list_orch))}")
print(f"Inference time Min: {np.min(np.asarray(inference_time_list_orch))}, Max: {np.max(np.asarray(inference_time_list_orch))}, Mean: {np.mean(np.asarray(inference_time_list_orch))}")
print("\n")

print(len(met_score_list))
print(len(met_score_list_orch))

Invalid count: 72
Arch Results
MET SCORE Min: 0.0, Max: 0.7765781836577119, Mean:0.29980824731041417, Median:0.2849370779269984
Inference time Min: 4.007554054260254, Max: 10.700841903686523, Mean: 7.578721990897542


Baseline Results 
MET SCORE Min: 0.12622720897615708, Max: 0.9133282942806752, Mean:0.38314935313883924, Median:0.35733475241580903
Inference time Min: 4.240699529647827, Max: 20.5083270072937, Mean: 10.0523871162108


1008
1008


In [9]:
for index in sorted(line_number_invalid, reverse=True):
    del met_score_list_orch[index]
    del inference_time_list_orch[index]